In [364]:
import numpy 

In [365]:
#Code parameters

nx = 81
dx = .25
dt = .0002   
gamma = 1.4

# all caculations at time = 0.01s
t = 0.01 
nt = int(t/dt)+1

In [366]:
# x=−10m to x=10m
x = numpy.linspace(-10, 10, nx)
rho = numpy.ones(nx)
rho[40:] = 0.125

u = numpy.zeros(nx)

p = numpy.ones(nx)
p[0:40] = 100000.
p[40:] = 10000.

In [367]:
#define u
eT = p/(gamma-1)/rho + 0.5*u**2
u1 = rho
u2 = rho*u
u3 = rho*eT

In [368]:
#define f mainly for the usage of following fhalf
def f1_func(u1, u2, u3):
    return u2
def f2_func(u1, u2, u3):
    return u2**2/u1 + (gamma-1)*(u3 - 0.5*u2**2/u1)
def f3_func(u1, u2, u3):
    return (u3 + (gamma - 1)*(u3 - 0.5*u2**2/u1))*u2/u1

In [381]:
def func(u1, u2, u3, dx, dt, nx, nt):
    un1 = numpy.zeros((nt, nx))
    un2 = numpy.zeros((nt, nx))
    un3 = numpy.zeros((nt, nx))
    
    un1[:,:] = u1.copy()
    un2[:,:] = u2.copy()
    un3[:,:] = u3.copy()
    
    uhalf1 = numpy.zeros(nx)
    uhalf2 = numpy.zeros(nx)
    uhalf3 = numpy.zeros(nx)
    
    uhalf1 = u1.copy()
    uhalf2 = u2.copy()
    uhalf3 = u3.copy()
    
    for i in range(1, nt):
        f1 = f1_func(u1, u2, u3)
        f2 = f2_func(u1, u2, u3)
        f3 = f3_func(u1, u2, u3)
        
        #get the half step of the function first 
        uhalf1[:-1] = 0.5*(u1[:-1] + u1[1:]) - (dt/(2*dx))*(f1[1:] - f1[:-1])
        uhalf2[:-1] = 0.5*(u2[:-1] + u2[1:]) - (dt/(2*dx))*(f2[1:] - f2[:-1])
        uhalf3[:-1] = 0.5*(u3[:-1] + u3[1:]) - (dt/(2*dx))*(f3[1:] - f3[:-1])
        
        #use the un half to calculate fn half 
        fhalf1 = f1_func(uhalf1, uhalf2, uhalf3)
        fhalf2 = f2_func(uhalf1, uhalf2, uhalf3)
        fhalf3 = f3_func(uhalf1, uhalf2, uhalf3)
        
        #the third step use the fn half to get un+1
        un1[i,1:] = u1[1:] - (dt/dx)*(fhalf1[1:] - fhalf1[:-1])
        un2[i,1:] = u2[1:] - (dt/dx)*(fhalf2[1:] - fhalf2[:-1])
        un3[i,1:] = u3[1:] - (dt/dx)*(fhalf3[1:] - fhalf3[:-1])
        
        u1 = un1[i].copy()
        u2 = un2[i].copy()
        u3 = un3[i].copy()
        
       
    return un1, un2, un3



        #I tried to combine 3 equ in to one func but failed how to express the time step?
        #Uhalf[:-1] = 0.5*(U[:-1] + U[1:]) - (dt/(2*dx))*(F[1:] - F[:-1])
        #Fhalf = [rho*Uhalf, rho*Uhalf**2+p, (rho*eT+p)*Uhalf]
        #Un[1:] = U[1:] - (dt/dx)*(Fhalf[1:] - Fhalf[:-1])


In [382]:
un = func(u1, u2, u3, dx, dt, nx, nt)

In [383]:
#first line is rho, and we need time at 0.01s and x=2.5m
rho = un[0][int(t/dt),int((10+2.5)/dx)]
#second line is rho*u, and we need time at 0.01s and x=2.5m
rhou = un[1][int(t/dt),int((10+2.5)/dx)]
#third line is rho*eT, and we need time at 0.01s and x=2.5m
rhoeT = un[2][int(t/dt),int((10+2.5)/dx)]

In [384]:
u = rhou/rho
print('At x=2.5m, velocity is {:.2f} m/s'.format(u))

At x=2.5m, velocity is 292.61 kg/m^3


In [385]:
eT = rhoeT/rho
p = (rho*eT - rho*0.5*u**2)*(gamma-1)
print('At x=2.5m, pressure is {:.2f} N/m^2'.format(p))

At x=2.5m, pressure is 30250.89 N/m^2


In [386]:
print('At x=2.5m,density is {:.2f} kg/m^3'.format(rho))

At x=2.5m,density is 0.37 kg/m^3
